In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans
import random

In [2]:
p1_data = pd.read_csv('data/P1.txt', delimiter=' ')

In [3]:
p1_data

,number,x,y,demand
0,1,34,-11,17
1,2,29,-35,17
2,3,15,0,10
3,4,-20,-24,4
4,5,-8,-21,7
...,...,...,...,...
95,96,-24,8,13
96,97,32,-33,17
97,98,15,21,14
98,99,-16,35,15


In [4]:
DEPOT_LOCATION = (-14, 9)

In [13]:
def without_replacement_sampling(array):
    sampled_value = random.sample(array, 1)
    array.remove(sampled_value[0])

    return sampled_value[0]

In [4]:
# evaluate_fitness_distance(population, p1_data, DEPOT_LOCATION )

In [380]:
def cross_over(chromosome1, chromosome2, depot_symbol=['***'], double_point=False):
    """
    single or double point cross-over for two chromosomes
    in this cross over, the corss over will be applied on raw chromosomes without depot (the depot symbols will be removed)
    then the time that needed to go back to depot the depot symbol will be inserted in genes

    Parameters:
    ------------
    chromosome1 : string
        the genes representing one vehicle
    chromosome2 : string
        the genes representing one vehicle
    depot_symbol : string
        if we have multiple depots
        then there would be different symbols for each
    double_point : bool
        if True, then the cross-over will be double point
        default is False
    
    
    Returns:
    ---------
    offspring1 : string
        the new chromsome created in cross-over phase
    offspring2: string
        the new chromsome created in cross-over phase
    """
    ## intialize the variables
    offspring1 = None
    offspring2 = None

    ## removing the depot genes from each chromosome
    parent1 = create_raw_chromsome(chromosome1, depot_symbol)
    parent2 = create_raw_chromsome(chromosome2, depot_symbol)
    
    ## possible indices to break the chromsome
    ## since each 3 characters represent one phenotype, then breaking gene from anywhere is not possible
    break_ind = random.randrange(3, 31, 3)
    break_ind2 = None

    if double_point:
        ## create another indice
        break_ind2 = random.randrange(3, 31, 3)
        ## to make sure two break points are not the same
        while break_ind2 == break_ind:
            break_ind2 = random.randrange(3, 31, 3)
    
        offspring1, offspring2 = break_double_point_gene(parent1, parent2, break_ind, break_ind2)
    else:
        offspring1, offspring2 = break_single_point_gene(parent1, parent2, break_ind)

    return offspring1, offspring2

            

def break_double_point_gene(chromosome1, chromosome2, ind1, ind2):
    """
    break the chromosome using double points
    """

    ## find the first and second breaking point of chromosome
    min_indice = min(ind1, ind2)
    max_indice = max(ind1, ind2)

    offspring1 = chromosome1[:min_indice] + chromosome2[min_indice:max_indice] + chromosome1[max_indice:]
    offspring2 = chromosome2[:min_indice] + chromosome1[min_indice:max_indice] + chromosome2[max_indice:]
    
    return offspring1, offspring2

def break_single_point_gene(chromosome1, chromosome2, indice):
    """
    break the chromosome using single points
    """

    ## find the first and second breaking point of chromosome
    offspring1 = chromosome1[:indice] + chromosome2[indice:]
    offspring2 = chromosome2[:indice] + chromosome1[indice:]
    
    return offspring1, offspring2

def create_raw_chromsome(chromosome, depot_symbols):
    """
    create a raw chromosome by removing the depots in its genes
    """
    ## the first and last chromsome's gene are the depots
    raw_chromomsome = chromosome[1:-1]
    
    ## removing the depot symbols
    for symbol in depot_symbols:
        raw_chromomsome = raw_chromomsome.replace(symbol, '')
    
    return raw_chromomsome

def post_process_capacity_based_chromsome(raw_chromsome, dataset, max_capacity, start_end_depot_symbol = '*', depot_symbol = '***'):
    """
    add the depot symbols to the chromsome using capacities
    if the vehicle (chromsome) does not have more capacity then adding the depot symbol to it
    means that the vehicle has gone back to depot and then served other next customers

    Important Notice: the depot symbol here is assumed as `*` for the starting point and ending point, and `***` for the middle ones 

    Parameters:
    ------------
    raw_chromsome : string
        a string representing the order of vechile serving
    dataset : dataframe
        getting the demand of each customer in it (`demand` column should be included with the customer number)
    max_capacity : int
        the maximum capacity that a vehicle can carry
    start_end_depot_symbol : string
        the start and the end of chromsome symbols representing the depot
    depot_symbol : string
        the depot symbol that is being used in the middle of the chromosomes
    
    Returns:
    ---------
    chromosome : string
        the chromsome with added depot symbols
    """
    tot_capacity = 0
    i = 3
    chromsome = ''
    # for i in range(0, len(raw_chromsome), 3):
    while i < len(raw_chromsome)+3:
        if raw_chromsome[i-3:i]:
            customer_number = int(raw_chromsome[i-3:i]) - 100
            customer = dataset[dataset.number == customer_number]

            tot_capacity += customer.demand.values[0]
        
            ## if the total capacity has gone more than the vehicle capacity, then add the depot symbol and 
            if tot_capacity > max_capacity:
                chromsome += depot_symbol
                ## reset the capacity
                tot_capacity = 0
            ## else add the customer number to the gene
            else:
                chromsome += raw_chromsome[i-3:i]
                ## go for the next customer (increase `i`) 
                i += 3

    ## start and the end of chromsome should be the depots
    chromsome = start_end_depot_symbol + chromsome + start_end_depot_symbol
        
    return chromsome

def post_process_capacity_based_population(raw_population, dataset, max_capacity ,start_end_depot_symbol = '*', depot_symbol = '***'):
    """
    post process the raw population based on their capacity 
    raw_population means population of chromsomes without any depot symbols

    Parameters:
    ------------
    raw_population : array of strings
        array of raw chromsomes
    dataset : dataframe
        getting the demand of each customer in it (`demand` column should be included with the customer number)
    start_end_depot_symbol : string
        the start and the end of chromsome symbols representing the depot
    depot_symbol : string
        the depot symbol that is being used in the middle of the chromosomes
    
    Returns:
    ---------
    population : array of strings
        the array of populations with depot symbols in it
    """
    population = []

    for raw_chromsome in raw_population:
        chromsome = post_process_capacity_based_chromsome(raw_chromsome, dataset, max_capacity, start_end_depot_symbol, depot_symbol)
        population.append(chromsome)
    
    return population

In [381]:
pop = cross_over(population[0], population[1])
pop

('163138170102106129143128116113', '159150189119158186117141165175')

In [383]:
pop2 = cross_over(population[0], population[1], double_point=True)
pop2

('163138170102158129143128116113', '159150189119106186117141165175')

# The method
## With Capacity Limit
1. **Representation** 
TODO: create shuffled customers in one string <br>
then add the deposit into the string based on capacities <br>
finally add `|` symbol to the string in the places before or after the depot_symbol to show different vehicles <br>
fitness is based on distances gone <br>

2. **Recombination**
TODO: Use the premutation methods <br>
and by that, recalculate the depots: make sure the capacities for vehicles are correct, <br>
if not correct recalculate the capacity and the routings for the next parts of the chromsome <br>

## With Distance Limit
1. all the work is the same but use the distance metric instead of capacity 

In [32]:

def generate_vehicles_chromosomes(dataset, MAX_CAPACITY, depot_symbol='(1)'):
    """
    Generate chromosomes of vehicles from 100 data point (data points are summed with 100 in order to be able to know whether the genes in chromosome )

    """

    arr = np.linspace(1, 100, 100, dtype=int)
    arr = list(arr)

    gene = depot_symbol
    capacity = 0

    while len(arr) > 0:

        customer_number = without_replacement_sampling(arr)
        data = dataset[dataset.number == customer_number]
        capacity += data.demand.values[0]

        ## if couldn't carry item, return to depot
        if capacity >= MAX_CAPACITY:
            gene += depot_symbol
            ## reset the capacity
            capacity = 0

            ## bring back the customer to our array
            ## since we haven't used it
            arr.append(data.number.values[0])

        else:
            gene += str(data.number.values[0] + 100) 
    gene += depot_symbol if gene[-1] != depot_symbol else ''
    
    return gene

def divide_vehicles(chromosome, vehicle_count, depot_symbol):
    """
    add the `|` as a division for a chromsome to make different vehicles
    """

    ## division point is the depots
    ## one less vehicle count should be the division points
    sample_arr = np.linspace(1, chromosome.count(depot_symbol) - 2, chromosome.count(depot_symbol) - 1, dtype=int )
    sample_arr = list(sample_arr)
    division_point = []
    for _ in range(vehicle_count - 1):
        point = without_replacement_sampling(sample_arr)
        division_point.append(point)
        
    ## making the copy of string
    vehicle_chromsome = chromosome

    for point in division_point:
        ## the process of finding point-th depot in chromsome
        occurance = -1
        occurance_idx = 0
        while occurance < point:
            occurance_idx = vehicle_chromsome.find(depot_symbol, occurance_idx) + 1
            occurance += 1
        
        vehicle_chromsome = vehicle_chromsome[:occurance_idx+2] + '|' + vehicle_chromsome[occurance_idx+2:]

    return vehicle_chromsome

def without_replacement_sampling(array):
    """
    get some value without replacement from array
    """
    sampled_value = random.sample(array, 1)
    array.remove(sampled_value[0])

    return sampled_value[0]

def process_division_points(vehicle_chromosome, depot_symbol):
    """
    Add the depot symbol after the division points
    """
    processed_vehicle_chromsome = vehicle_chromosome

    division_counts = vehicle_chromosome.count('|')

    for idx in range(division_counts):
        occurance = -1
        occurance_idx = 0
        while occurance < idx:
            occurance_idx = processed_vehicle_chromsome.find('|', occurance_idx) + 1
            occurance += 1
        
        processed_vehicle_chromsome = processed_vehicle_chromsome[:occurance_idx] + depot_symbol + processed_vehicle_chromsome[occurance_idx:]
    
    return processed_vehicle_chromsome


def evaluate_distance_fitness(chromsome, DEPOT_LOCATION, dataset):
    """
    evaluate the distance gone for a chromsome containing different vehicle with the splitter symbol `|`

    """

    depot_x_loc, depot_y_loc = DEPOT_LOCATION

    distance = 0
    for ch in chromsome.split('(1)'):
        if ch and ch != '|':
            ## start is always from a depot
            last_loc_X, last_loc_Y = DEPOT_LOCATION
            for i in range(3, len(ch)+1, 3):
                ## customer number
                customer_no = int(ch[i-3:i]) - 100

                ## finding the exact customer location from the dataset            
                customer = dataset[dataset.number == customer_no]
                customer_X = customer.x.values[0]
                customer_Y = customer.y.values[0]

                ## manhatan distance
                distance += abs(last_loc_X - customer_X) + abs(last_loc_Y - customer_Y)
                ## update the last locations
                last_loc_X, last_loc_Y = customer_X, customer_Y

            ## end is always the depot
            distance += abs(last_loc_X - depot_x_loc) + abs(last_loc_Y - depot_y_loc)
    
    return distance

def generate_population(max_capacity, DEPOT_LOCATION, dataset, depot_symbol = '(1)', pop_count = 10, vehicle_count=6):
    """
    generate the population of the problem

    Parameters:
    ------------
    depot_symbol : string
        the symbol for the depot
    max_capacity : int
        maximum capacity of a vehicle
    dataset : dataframe
        the whole information about the problem

    Returns:
    ----------
    population_arr : array
        the array of different chromsomes
    fitness_arr : array
        the fitnesses corresponding to each chromsome
    """
    ## chromsomes array
    population_arr = []

    ## fitness array
    fitness_arr = []

    for _ in range(pop_count):
        
        ## just make chromsomes with depot symbols
        chromsome = generate_vehicles_chromosomes(dataset, max_capacity, depot_symbol)
        ## divide it into different part to make it like different vehicles
        vehicle_chromsome = divide_vehicles(chromsome, vehicle_count, depot_symbol)
        ## process the divisions and make sure that the start points has the depot symbol 
        processed_vehicle_chromsome = process_division_points(vehicle_chromsome, depot_symbol)
        
        ## add to population array
        population_arr.append(processed_vehicle_chromsome)

        ## fitness evaluations
        chromsome_fitness = evaluate_distance_fitness(processed_vehicle_chromsome, DEPOT_LOCATION, dataset)
        fitness_arr.append(chromsome_fitness)
        
    return population_arr, fitness_arr

In [51]:
def process_depots(dataset ,string_chromsome_arr, max_capacity, depot_symbol):
    """
    Add the depot symbols to string chromsome array (the customer numbers are strings in the array)
    """
    capacity = 0
    idx = 0
    string_chromsome = depot_symbol
    while idx < len(string_chromsome_arr):
        gene = string_chromsome_arr[idx]

        customer_no = int(gene) - 100
        customer = dataset[dataset.number == customer_no]

        capacity += customer.demand.values[0]
        
        if capacity > max_capacity:
            string_chromsome += depot_symbol
            capacity = 0
        else:
            string_chromsome += gene
            idx += 1

    string_chromsome += depot_symbol if string_chromsome[-3:] != depot_symbol else ''
    
    
    return string_chromsome


# Starting

In [5]:
from generate_population_scripts import generate_population

In [6]:
population_array, fitness_array = generate_population(70, (-14, 9), p1_data)

'(1)'

In [33]:
evaluate_distance_fitness(population_array[0], DEPOT_LOCATION, p1_data)

8112

In [53]:
U_chromsome = split_string_step(population_array[0].replace('(1)', '').replace('|', ''), 3)
U_chromsome = process_depots(p1_data, U_chromsome, 70, '(1)')
U_chromsome

'(1)197112153200135116121(1)150111145125137(1)152127158134105160(1)122139142198(1)138101131167123(1)199168181129104151180172(1)130186144120184(1)179103(1)193185166176(1)115107119191156190(1)136169117106157(1)194173196170147(1)149159165171(1)141188162143(1)161114178146126187(1)140189102195(1)174164148124(1)108175132113133109(1)118183182110128192177155(1)163154(1)'

In [54]:
divide_vehicles(U_chromsome, 6, '(1)')

'(1)197112153200135116121(1)150111145125137(1)152127158134105160(1)122139142198(1)|138101131167123(1)199168181129104151180172(1)130186144120184(1)179103(1)193185166176(1)|115107119191156190(1)136169117106157(1)194173196170147(1)149159165171(1)141188162143(1)161114178146126187(1)|140189102195(1)174164148124(1)108175132113133109(1)|118183182110128192177155(1)|163154(1)'

In [105]:
def split_string_step(string, step):
    """
    split a string based on steps
    """
    string_arr = []

    for i in range(1, len(string)+1):
        if i % step == 0:
            string_arr.append(string[i-step:i])
    return string_arr

def cut_and_crossfil(chromosome1, chromosome2, dataset, max_capacity, depot_symbol=['(1)']):
    """
    cut and crossfill cross-over

    at the end repair the chromsome with max_capacity
    """
    ## find the vehicle count
    vehicle_count = chromosome1.count('|') + 1

    ## find the chromsome with minimum length and break it
    chromosome_to_break = chromosome1 if len(chromosome1) < len(chromosome2) else chromosome2
    second_chromsome = chromosome1 if len(chromosome1) > len(chromosome2) else chromosome2

    ## remove all the symbols inside the chromsome
    chromosome = chromosome_to_break.replace('|', '')
    second_chromsome = second_chromsome.replace('|', '')

    for symbol in depot_symbol:
        chromosome = chromosome.replace(symbol, '')
        second_chromsome = second_chromsome.replace(symbol, '')
    
    ## break point of the chromsomes
    ## the break point should not be the first or the last points
    ## create an array and randomly sample from it
    arr = np.arange(3, min(len(chromosome), len(second_chromsome)) - 3, 3)
    break_point = without_replacement_sampling(list(arr))
    break_point = break_point // 3
    
    chromosome_arr = split_string_step(chromosome, 3)
    second_chromsome_arr = split_string_step(second_chromsome, 3)

    ## offspring creation
    offspring1 = chromosome_arr[:break_point]

    for gene in second_chromsome_arr[break_point:] + second_chromsome_arr[:break_point]:
        if gene not in offspring1:
            offspring1.append(gene)
    
    offspring2 = second_chromsome_arr[break_point:]
    for gene in chromosome_arr[:break_point] + chromosome_arr[break_point:] :
        if gene not in offspring2:
            offspring2.append(gene)

    ## repairment
    random_depot = random.sample(depot_symbol, 1)[0]
    offspring1 = process_depots(dataset, offspring1, max_capacity, random_depot)
    offspring1 = divide_vehicles(offspring1, vehicle_count, random_depot)
    offspring1 = process_division_points(offspring1, random_depot)

    random_depot = random.sample(depot_symbol, 1)[0]
    offspring2 = process_depots(dataset, offspring2, max_capacity, random_depot)
    offspring2 = divide_vehicles(offspring2, vehicle_count, random_depot)
    offspring2 = process_division_points(offspring2, random_depot)



    return offspring1, offspring2

In [106]:
U_offsprings = cut_and_crossfil(population_array[0], population_array[1], p1_data, 70)

In [104]:
A = ['1', '2']
B = ['4', '5']
A + B

['1', '2', '4', '5']

In [31]:
'|'.join(A_U)

'134|678'

In [ ]:
## TODO: Selection method
## TODO: replacement method